In [ ]:
# Decision Trees
# Support Vector Machine

#Potential Dataset - https://www.kaggle.com/mbornoe/lisa-traffic-light-dataset
#https://www.kaggle.com/cactus3/basicshapes
#https://www.kaggle.com/sanikamal/rock-paper-scissors-dataset

###FAKE NEWS
# https://www.kaggle.com/c/fake-news/data
# https://ieee-dataport.org/open-access/fnid-fake-news-inference-dataset#files

###FASHION
#https://www.kaggle.com/dqmonn/zalando-store-crawl
#https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069 https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069 https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069 https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069 https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069 https://www.kaggle.com/faressayah/fashion-mnist-classification-using-cnns?scriptVersionId=54319069

##INSTRUCTIONS
#pre-processing: https://www.kaggle.com/matleonard/intro-to-nlp
#literally what we're doing but with different labels: https://www.kaggle.com/matleonard/text-classification

In [ ]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

url = 'https://drive.google.com/file/d/1hazpONaxqZyTbvy76WWCo1s9QnvRsFHg/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path); #Read csv into dataframe
df = df.replace({"label":0},"reliable").replace({"label":1},"unreliable");
df.head() #Print out first 5 values
print("hello")
traindf, testdf = train_test_split(df, test_size = 0.2)
print('done splitting')

hello
done splitting


In [5]:
import spacy
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
nlp = spacy.blank("en");
config = {
   "threshold": 0.5,
   "model": DEFAULT_SINGLE_TEXTCAT_MODEL,
}
textcat = nlp.add_pipe('textcat',config=config);
textcat.add_label("reliable");
textcat.add_label("unreliable");

t_text = traindf['text'].values;
t_labels = [{'cats':{'reliable': label == 'reliable', 'unreliable': label == 'unreliable'}} for label in traindf['label']];
training_data = list(zip(t_text, t_labels))
print('done')

done


In [6]:
from spacy.util import minibatch
from spacy.training import Example
import random

random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(5):
    random.shuffle(training_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(training_data, size=20)
    # Iterate through minibatches
    count = 0
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(str(text))
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
        count+=1
        if (count%10 == 0):
            print("10 batch: ",count)
    print(losses)
    

10 batch:  10
10 batch:  20


/Users/jhan/opt/anaconda3/lib/python3.8/site-packages/thinc/layers/layernorm.py:32: RuntimeWarning: invalid value encountered in reciprocal
  d_xhat = N * dY - sum_dy - dist * var ** (-1.0) * sum_dy_dist


10 batch:  30
10 batch:  40
10 batch:  50
10 batch:  60
10 batch:  70
10 batch:  80
10 batch:  90
10 batch:  100
10 batch:  110
10 batch:  120
10 batch:  130
10 batch:  140
10 batch:  150
10 batch:  160
10 batch:  170
10 batch:  180
10 batch:  190
10 batch:  200
10 batch:  210
10 batch:  220
10 batch:  230
10 batch:  240
10 batch:  250
10 batch:  260
10 batch:  270
10 batch:  280
10 batch:  290
10 batch:  300
10 batch:  310
10 batch:  320
10 batch:  330
10 batch:  340
10 batch:  350
10 batch:  360
10 batch:  370
10 batch:  380
10 batch:  390
10 batch:  400
10 batch:  410
10 batch:  420
10 batch:  430
10 batch:  440
10 batch:  450
10 batch:  460
10 batch:  470
10 batch:  480
10 batch:  490
10 batch:  500
10 batch:  510
10 batch:  520
10 batch:  530
10 batch:  540
10 batch:  550
10 batch:  560
10 batch:  570
10 batch:  580
10 batch:  590
10 batch:  600
10 batch:  610
10 batch:  620
10 batch:  630
10 batch:  640
10 batch:  650
10 batch:  660
10 batch:  670
10 batch:  680
10 batch:  690
10

In [7]:
from IPython.display import display

display(testdf.tail())
texts = testdf['text'].values;
print('done')

,id,title,author,text,label
1665,1665,"Aleppo Evacuation Effort Restarts, and Assad C...",Ben Hubbard and Hwaida Saad,"BEIRUT, Lebanon — After months of fierce bo...",reliable
5615,5615,Feinstein: Trump ’Splitting’ America Apart Wit...,Pam Key,On this weekend’s broadcast of “Fox News Sunda...,reliable
7787,7787,Hot Mic: DHS Chief John Kelly Jokes Trump Can ...,Katie McHugh,Department of Homeland Security Secretary John...,reliable
10038,10038,Markets Do Not Want Trump,dailouk,Home | World | Markets Do Not Want Trump Marke...,unreliable
3654,3654,Review: ‘Absolutely Fabulous’ Brings Its Snick...,Manohla Dargis,“Absolutely Fabulous: The Movie” opens with Ed...,reliable


done


In [9]:
docs = [nlp.tokenizer(str(text)) for text in texts]
print('done');

done


In [11]:
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)
print(type(scores))

<class 'numpy.ndarray'>


In [12]:
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])
print(predicted_labels)

['unreliable', 'unreliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'reliable', 'unreliable', 'reliable', 'reliable', 'unreliable', 'unreliable', 'unreliable', 'unreliable', 'reliable', 'reliable', 'reliable', 'unreliable', 'reliable', 'unreliable', 'reliable', 'unreliable', 'reliable', 'unreliable', 'unreliable', 'reliable', 'unreliable', 'reliable', 'unre

In [20]:
correct = 0
total = 5200
labels = testdf['label'].values
print(labels)
for i in range (4160):
    if(labels[i] == textcat.labels[predicted_labels[i]]):
        correct += 1
print(correct, '/', '4160')

['unreliable' 'unreliable' 'unreliable' ... 'reliable' 'unreliable'
 'reliable']
4145 / 4160


In [21]:
import pickle 
filename = 'finalized_model.sav'
with open('nlp_trained2', 'wb') as files:
    pickle.dump(nlp, files)
print("run");

run


In [ ]:
with open('nlp_trained' , 'rb') as f:
     model = pickle.load(f)